In [5]:
from pathlib import Path
import pandas as pd
import numpy as np

from chan.signals.mmd import (
    DEFAULT_POLICY, _map_beichi_to_1st,
    generate_mmd2_from_mmd1, generate_mmd3_from_zs,
    merge_mmd_signals
)
from chan.signals.merge import generate_trading_signals

ROOT = Path.cwd().resolve()
if ROOT.name == "Note": ROOT = ROOT.parents[0]
DATA = ROOT / "DATA"
CHAN = ROOT / "chan_data"

symbols = ["BTCUSDT","ETHUSDT"]
tfs = ["5m","15m","30m","1h","4h"]

def _read_k(sym, tf):
    df = pd.read_csv(DATA/sym/f"{tf}.csv")
    if "timestamp" not in df.columns:
        for c in ("time","datetime","ts"):
            if c in df.columns:
                df = df.rename(columns={c:"timestamp"})
                break
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True, errors="coerce")
    df = df.dropna(subset=["timestamp"]).sort_values("timestamp").reset_index(drop=True)
    return df

def _read_beichi(sym, tf):
    p = CHAN/"signals"/"beichi"/sym/f"{tf}_beichi_all.csv"
    df = pd.read_csv(p, parse_dates=["ts"])
    df["ts"] = pd.to_datetime(df["ts"], utc=True, errors="coerce")
    return df

def _read_segments(sym, tf):
    p = CHAN/"segments"/"fast"/sym/f"{tf}_segments.csv"
    df = pd.read_csv(p, parse_dates=["timestamp_start","timestamp_end"])
    df["timestamp_start"] = pd.to_datetime(df["timestamp_start"], utc=True, errors="coerce")
    df["timestamp_end"]   = pd.to_datetime(df["timestamp_end"],   utc=True, errors="coerce")
    return df

def _read_zs(sym, tf):
    p = CHAN/"zhongshu_hierarchy_fast"/sym/f"{tf}_L1.csv"
    if not p.exists():
        return pd.DataFrame()
    df = pd.read_csv(p, parse_dates=["timestamp_start","timestamp_end"])
    df["timestamp_start"] = pd.to_datetime(df["timestamp_start"], utc=True, errors="coerce")
    df["timestamp_end"]   = pd.to_datetime(df["timestamp_end"],   utc=True, errors="coerce")
    return df


for sym in symbols:
    mmd_by_tf = {}
    zs_by_tf  = {}
    for tf in tfs:
        k   = _read_k(sym, tf)
        bc  = _read_beichi(sym, tf)
        seg = _read_segments(sym, tf)
        zs  = _read_zs(sym, tf)
        zs_by_tf[tf] = zs

        mmd1 = _map_beichi_to_1st(bc, k, dict(DEFAULT_POLICY))
        mmd2 = generate_mmd2_from_mmd1(k, mmd1, dict(DEFAULT_POLICY))
        mmd3 = generate_mmd3_from_zs(k, seg, zs, dict(DEFAULT_POLICY))
        mmd  = merge_mmd_signals(mmd1, mmd2, mmd3)
        mmd_by_tf[tf] = mmd

        out_dir = CHAN/"signals"/"mmd"/sym
        out_dir.mkdir(parents=True, exist_ok=True)
        mmd.to_csv(out_dir/f"{tf}_mmd123.csv", index=False)
        print(f"[MMD] {sym}-{tf}: 1={len(mmd1)}, 2={len(mmd2)}, 3={len(mmd3)}, all={len(mmd)}")

    merged = generate_trading_signals(mmd_by_tf, zs_by_tf)
    out2 = CHAN/"signals"/"trading"/sym
    out2.mkdir(parents=True, exist_ok=True)
    merged.to_csv(out2/"multi_tf_trading_signals.csv", index=False)
    print(f"[TRD] {sym}: {len(merged)} -> {out2/'multi_tf_trading_signals.csv'}")


[MMD] BTCUSDT-5m: 1=29, 2=10, 3=28, all=67
[MMD] BTCUSDT-15m: 1=29, 2=6, 3=35, all=70
[MMD] BTCUSDT-30m: 1=30, 2=7, 3=32, all=69
[MMD] BTCUSDT-1h: 1=19, 2=5, 3=24, all=48
[MMD] BTCUSDT-4h: 1=3, 2=2, 3=7, all=12
[TRD] BTCUSDT: 259 -> /Users/cc/PycharmProjects/ChanLab/chan_data/signals/trading/BTCUSDT/multi_tf_trading_signals.csv
[MMD] ETHUSDT-5m: 1=123, 2=36, 3=119, all=278
[MMD] ETHUSDT-15m: 1=63, 2=20, 3=84, all=167
[MMD] ETHUSDT-30m: 1=41, 2=15, 3=48, all=104
[MMD] ETHUSDT-1h: 1=11, 2=2, 3=21, all=34
[MMD] ETHUSDT-4h: 1=6, 2=3, 3=4, all=13
[TRD] ETHUSDT: 587 -> /Users/cc/PycharmProjects/ChanLab/chan_data/signals/trading/ETHUSDT/multi_tf_trading_signals.csv


In [9]:
from pathlib import Path
import pandas as pd

ROOT = Path.cwd().resolve()
if ROOT.name == "Note": ROOT = ROOT.parents[0]
CHAN = ROOT / "chan_data"

rows = []
for p in CHAN.rglob("*.csv"):
    try:
        rel = p.relative_to(CHAN)
    except Exception:
        rel = p.name
    rows.append(dict(
        rel=str(rel),
        bytes=p.stat().st_size,
        mtime=pd.Timestamp(p.stat().st_mtime, unit="s", tz="UTC")
    ))

mf = pd.DataFrame(rows).sort_values("rel").reset_index(drop=True)
outp = CHAN / "_manifest.csv"
mf.to_csv(outp, index=False)
print(f"[MANIFEST] {len(mf)} files -> {outp}")
print(mf.head(10))

[MANIFEST] 168 files -> /Users/cc/PycharmProjects/ChanLab/chan_data/_manifest.csv
                     rel    bytes                               mtime
0  BTCUSDT/15m_chanK.csv  1213893 2025-10-14 02:32:27.162913084+00:00
1   BTCUSDT/1h_chanK.csv   301113 2025-10-14 02:32:29.096009254+00:00
2  BTCUSDT/30m_chanK.csv   605207 2025-10-14 02:32:28.460666895+00:00
3   BTCUSDT/4h_chanK.csv    69923 2025-10-14 02:32:29.264950514+00:00
4   BTCUSDT/5m_chanK.csv  3600643 2025-10-14 02:32:24.363869190+00:00
5  ETHUSDT/15m_chanK.csv  1121010 2025-10-14 02:32:39.931023598+00:00
6   ETHUSDT/1h_chanK.csv   267295 2025-10-14 02:32:41.785378218+00:00
7  ETHUSDT/30m_chanK.csv   547712 2025-10-14 02:32:41.145197630+00:00
8   ETHUSDT/4h_chanK.csv    65633 2025-10-14 02:32:41.938349009+00:00
9   ETHUSDT/5m_chanK.csv  3396132 2025-10-14 02:32:37.347111225+00:00
